In [ ]:
! pip install langchain_community langchain_openai faiss_cpu

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import CharacterTextSplitter
from google.colab import userdata
import os
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor, DocumentCompressorPipeline
from langchain_openai import OpenAI, ChatOpenAI, OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.retrievers.document_compressors import LLMChainFilter
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_text_splitters import CharacterTextSplitter


In [ ]:
documents = TextLoader("/content/state_of_the_union.txt").load()

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)

In [ ]:
texts = text_splitter.split_documents(documents)

In [ ]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

In [ ]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
retriever = FAISS.from_documents(texts, OpenAIEmbeddings()).as_retriever()

In [ ]:
docs = retriever.invoke("What did the president say about Ketanji Brown Jackson?")

In [ ]:
# Helper function for printing docs
 
def pretty_print_docs(docs):
    print( 
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [ ]:
pretty_print_docs(docs)

In [ ]:
query = "What were the top tree priorities in the most recent State of the Union address?"

In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [ ]:
print(chain.invoke(query)["text"])

In [ ]:
compressor = LLMChainExtractor.from_llm(llm)

In [ ]:
compressor_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

In [ ]:
compressor_docs = compressor_retriever.invoke(query)

In [ ]:
pretty_print_docs(compressor_docs)

In [ ]:
_filter = LLMChainFilter.from_llm(llm)

In [ ]:
compressor_retriever_2 = ContextualCompressionRetriever(base_compressor=_filter, base_retriever=retriever)

In [ ]:
compressor_docs_2 = compressor_retriever_2.invoke(query)

In [ ]:
pretty_print_docs(compressor_docs_2)

In [ ]:
original_contexts_len = len("\n\n".join([d.page_content for i, d in enumerate(docs)]))

In [ ]:
original_contexts_len

In [ ]:
compressor_contexts_len = len("\n\n".join([d.page_content for i, d in enumerate(compressor_docs)]))

In [ ]:
print("Compressed Ratio: ", f"{original_contexts_len/ (compressor_contexts_len + 1e-5):.2f}x")

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)

In [ ]:
compressor_retriever_3 = ContextualCompressionRetriever(base_compressor=embeddings_filter, base_retriever=retriever)

In [ ]:
pretty_print_docs(compressor_retriever_3)

In [ ]:
compressor_docs_3 = compressor_retriever_3.invoke(query)

In [ ]:
pretty_print_docs(compressor_docs_3)

In [ ]:
compressor_contexts_len_2 = len("\n\n".join([d.page_content for i, d in enumerate(compressor_docs_3)]))

In [ ]:
print("Compressed context length: ", compressor_contexts_len_2)

In [ ]:
print("Compressed Ratio: ", f"{original_contexts_len/ (compressor_contexts_len_2 + 1e-5):.2f}x")

In [ ]:
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0, separator=". ")

In [ ]:
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)

In [ ]:
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)

In [ ]:
pipeline_compressor = DocumentCompressorPipeline(transformers=[splitter, redundant_filter, redundant_filter])

In [ ]:
compressor_retriever_4 = ContextualCompressionRetriever(base_compressor=pipeline_compressor, base_retriever=retriever)

In [ ]:
compressor_docs_4 = compressor_retriever_4.invoke(query)

In [ ]:
pretty_print_docs(compressor_docs_4)

In [ ]:
chain_1 = RetrievalQA.from_chain_type(llm=llm, retriever=compressor_docs_4)

In [ ]:
chain_1.invoke(query)

In [ ]:
print(chain_1.invoke(query)["result"])